# Dataset Preprocessing

Build ... TODO

Download from ... TODO



In [53]:
import pandas as pd
import numpy as np

## Generate ratings.csv

Convert four behaviors into explicit ratings:

['click', 'collect', 'cart', 'alipay'] → [1.25, 2.5, 3.75, 5]

In [54]:
# Read raw data
log = pd.read_csv("./raw/(sample)sam_tianchi_2014002_rec_tmall_log.csv")
log.groupby('action').count()

,item_id,user_id,vtime
action,,,
alipay,7,7,7
cart,358,358,358
click,4474,4474,4474
collect,5,5,5


In [55]:
# Convert four behaviors into explicit ratings
log.action = log.action.replace(
    ['click', 'collect', 'cart', 'alipay'], 
    [1.25, 2.5, 3.75, 5])
log.groupby('action').count()

,item_id,user_id,vtime
action,,,
1.25,4474,4474,4474
2.50,5,5,5
3.75,358,358,358
5.00,7,7,7


In [56]:
# Save to ratings.csv
log.to_csv('ratings.csv', index=False, sep='\t')

## Generate kg.txt

**from `...log.csv`**

* action_pattern
* unique_user
* purchase_frequency

**from `...product.csv`**

* product.belong_to.leave_category
* leave_category.belong_to.parent_category
* 
* product.product_brand.brand
* product.selled_by.seller

**from `...review.csv`**

* feedback_pattern
* purchase_feedback_interval

In [57]:
product = pd.read_csv('./raw/(sample)sam_tianchi_2014001_rec_tmall_product.csv', encoding='GBK')
    # dtype={'title':'str', 'pict_url':'str', 'category':'str', 'brand_id':'str', 'seller_id':'str'}
product.dtypes

item_id       int64
title        object
pict_url     object
category     object
brand_id     object
seller_id    object
dtype: object

In [58]:
kg = [['head', 'relation', 'tail'], ]
for _, row in product.iterrows():
    
    # product.belong_to.leave_category
    kg.append([
        f'i{row.item_id}',
        'belong_to',
        f'c{row.category}'
    ])

    # leave_category.belong_to.parent_category
    entry = [
        f'c{row.category}',
        'belong_to',
        f'c{row.category.split("-")[0]}'
    ]
    kg.append(entry) if entry not in kg else ()

    # product.product_brand.brand
    kg.append([
        f'i{row.item_id}',
        'product_brand',
        f'{row.brand_id}'
    ]) if row.brand_id == row.brand_id else ()  # (NaN == NaN) => False

    # product.selled_by.seller
    kg.append([
        f'i{row.item_id}',
        'selled_by',
        f'{row.seller_id}'
    ])
np.savetxt('kg.txt', kg[1:], fmt='%s', delimiter='\t')

## Generate item_index2entity_id.txt

The items/entities include user id, product id, parent/leave category, brand id, seller id.

In [59]:
kg = np.array(kg)
items_in_kg = kg[1:, 0::2].flatten().tolist()
items_of_user = log.user_id.tolist()
items = items_in_kg + items_of_user
items = list(set(items))  # Move duplicates
i2e = [[item, entity] for entity, item in enumerate(items)]
np.savetxt('item_index2entity_id.txt', i2e, fmt='%s', delimiter='\t')